In [1]:
! wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-06-20 19:11:13--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-20 19:11:13 (9.18 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [4]:
! wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-20 19:15:51--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.04s   

2024-06-20 19:15:51 (16.2 MB/s) - ‘documents.json’ saved [658332/658332]



In [11]:
import minsearch
import json
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = "course-question"
es_client.indices.create(index = index_name, body = index_settings)
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index = index_name, document = doc)

In [4]:
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [5]:
with open('documents.json', 'r') as f:
    doc_raws = json.load(f)

In [6]:
documents = []
for course_dict in doc_raws:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
index = minsearch.Index(text_fields = ['question', 'text', 'section'],
                         keyword_fields = ['course']
                        )
index.fit(documents)

In [46]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    result  = index.search(query = query,
                 boost_dict = boost,
                 num_results = 5
                )
    return result

def elastic_search (query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    es_result = es_client.search(index = index_name, body=search_query)
    es_result_docs = []
    for hit in es_result['hits']['hits']:
        es_result_docs.append(hit['_source'])


    return es_result_docs
    
def build_prompt(query, search_result):

    context = ""
    for doc in search_result:
        context =context + f"section :{doc['section']} \nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"
    
    prompt_template = """
    You are a course teaching assistant. Answer the given QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}\n
    CONTEXT: {context}
    """.strip()
    prompt = prompt_template.format(question = query, 
                                    context = context).strip()
    return prompt

def llm (prompt):
    response = client.chat.completions.create(
    model = 'gpt-4o',
    messages =[{'role':'user', 'content': prompt}]
    )
    return response.choices[0].message.content

In [47]:
def rag(query):
    results = elastic_search(query)#search(query)
    prompt = build_prompt(query, results)
    answer = llm(prompt)
    return answer

In [50]:
query = "Sunrun is a great company. I am glad I went solar with Sunrun"
rag(query)

"I'm sorry, but as a course teaching assistant, I can only provide information related to the course based on the provided FAQs. If you have any questions about the course, modules, or specific topics, please let me know and I'd be happy to assist!"